In [3]:

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# NLTK Stop words
import nltk; 
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

import openpyxl

filename = "result_highlights_updateddata.xlsx"
wb = openpyxl.load_workbook(filename)
sheet = wb.active

data = []
for _ in range(87):
  index = _ + 1
  cell = sheet.cell(row = index, column = 1)
  data.append(cell.value)

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
#print(data_words[:1])

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#print(data_lemmatized[:1])

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(corpus[:1])

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/michael/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[(0,
  '0.023*"source" + 0.021*"know" + 0.020*"reliable" + 0.018*"look" + '
  '0.017*"make" + 0.015*"say" + 0.015*"go" + 0.014*"article" + 0.012*"think" + '
  '0.012*"thing"'),
 (1,
  '0.019*"would" + 0.015*"say" + 0.015*"article" + 0.014*"know" + 0.012*"see" '
  '+ 0.012*"quote" + 0.012*"able" + 0.011*"source" + 0.011*"university" + '
  '0.011*"first"'),
 (2,
  '0.035*"source" + 0.029*"look" + 0.026*"reliable" + 0.025*"seem" + '
  '0.019*"article" + 0.016*"really" + 0.015*"know" + 0.013*"pretty" + '
  '0.013*"probably" + 0.013*"go"'),
 (3,
  '0.020*"go" + 0.020*"look" + 0.016*"really" + 0.016*"article" + 0.015*"seem" '
  '+ 0.015*"say" + 0.015*"kind" + 0.013*"thing" + 0.013*"know" + 0.013*"link"'),
 (4,
  '0.002*"look" + 0.001*"article" + 0.001*"go" + 0.001*"see" + '
  '0.001*"reliable" + 0.001*"know" + 0.001*"make" + 0.001*"thing" + '
  '0.001*"really" + 0.001*"source"'),
 (5,
  '0.042*"reliable" + 0.031*"source" + 0.022*"think" + 0.019*"seem" + '
  '0.016*"make" + 0.010*"thing" + 0.

In [4]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/michael/miniconda3/envs/nlp/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.087768  0.002489       1        1  40.171799
0      0.113728 -0.008489       2        1  11.738886
16     0.153198  0.005245       3        1   9.093039
2      0.162864 -0.019027       4        1   8.787520
14     0.120423 -0.008457       5        1   6.192026
3      0.109275 -0.007227       6        1   4.728129
15     0.118889 -0.007021       7        1   3.904042
17     0.080665  0.029723       8        1   3.690675
19     0.077162 -0.037380       9        1   3.574229
6      0.063338 -0.028357      10        1   2.078302
11     0.030380  0.001504      11        1   1.551744
1     -0.001341  0.196165      12        1   1.517763
5      0.018580 -0.025657      13        1   1.244480
7      0.018910 -0.031808      14        1   1.190606
9     -0.093850 -0.032181      15        1   0.493866
10    -0.187757 -0.006591      16        1   0.028149
18    -0.200837 -0.005232      17        1   0.003760
4     -0.214299 -0.006274      18        1   0.003712
13    -0.227871 -0.006149      19        1   0.003673
12    -0.229225 -0.005276      20        1   0.003606, topic_info=    Category        Freq      Term       Total  loglift  logprob
85   Default  821.000000      know  821.000000  30.0000  30.0000
154  Default  935.000000  reliable  935.000000  29.0000  29.0000
176  Default  974.000000    source  974.000000  28.0000  28.0000
11   Default  931.000000   article  931.000000  27.0000  27.0000
94   Default  977.000000      look  977.000000  26.0000  26.0000
..       ...         ...       ...         ...      ...      ...
250  Topic20    0.000682       let  305.604980  -2.7825  -7.9560
213  Topic20    0.000675      well  254.949188  -2.6114  -7.9662
110  Topic20    0.000667      much  212.234528  -2.4397  -7.9779
7    Topic20    0.000678      also  306.579315  -2.7916  -7.9619
138  Topic20    0.000670    pretty  363.854462  -2.9754  -7.9745

[1496 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
360       1  0.400881   able
360       2  0.080176   able
360       3  0.120264   able
360       4  0.020044   able
360       5  0.080176   able
...     ...       ...    ...
1376     13  0.716381  yield
1679      1  0.281569    yup
1679      5  0.563138    yup
2691     10  0.494152    zoo
2691     15  0.494152    zoo

[3062 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 1, 17, 3, 15, 4, 16, 18, 20, 7, 12, 2, 6, 8, 10, 11, 19, 5, 14, 13])

In [5]:
mallet_path = "/Users/michael/Projects/elise/2019_oct/mallet-2.0.8/bin/mallet"
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

# Show Topics
pprint(ldamallet.show_topics(num_topics=20, num_words=10, formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(0,
  [('point', 0.05515898767034393),
   ('thing', 0.053212199870214145),
   ('question', 0.04023361453601557),
   ('show', 0.038286826735885786),
   ('information', 0.029850746268656716),
   ('cool', 0.029201817001946788),
   ('people', 0.027903958468526932),
   ('search', 0.02336145360155743),
   ('trust', 0.020765736534717714),
   ('put', 0.020765736534717714)]),
 (1,
  [('article', 0.07785888077858881),
   ('quote', 0.06472019464720194),
   ('source', 0.061800486618004864),
   ('give', 0.04476885644768856),
   ('link', 0.04184914841849149),
   ('statistic', 0.032116788321167884),
   ('study', 0.028223844282238442),
   ('direct', 0.024817518248175182),
   ('opinion', 0.01995133819951338),
   ('fairly', 0.01800486618004866)]),
 (2,
  [('article', 0.12261960055736182),
   ('author', 0.07292150487691593),
   ('write', 0.06409660938225732),
   ('quote', 0.048304691128657685),
   ('professional', 0.03855085926614027),
   ('study', 0.03204830469112866),
   ('research', 0.031119368323269